In [ ]:
import pandas as pd
import json

In [ ]:
suhi_10_29_2024_df = pd.read_csv("/content/SUHI_MATRIX_WITH_CHW_Notes_10_29_2024.csv")

In [ ]:
suhi_10_29_2024_df.columns

Index(['record_id', 'age', 'sum_contact_attempts_calc', 'contact_attempts',
       'day_readmit', 'total_time_spent', 'Visit_Number', 'Count_of_visit',
       'comorbidity', 'sumComorbidities',
       ...
       'resources_hiv_notes', 'resources_covid_notes',
       'resources_diabetes_notes', 'resources_asthma_notes',
       'resources_alc_notes', 'resources_substance_notes',
       'resources_emotional_notes', 'sdoh_other_notes',
       'resources_other_notes', 'sdoh_notes'],
      dtype='object', length=150)

In [ ]:
# Concatenate all the fields with if field name contains notes_contact into combined_notes
contact_note_cols = []
for col in suhi_10_29_2024_df.columns:
    if 'notes_contact' in col:
        contact_note_cols.append(col)

In [ ]:
# Concatenate all the cols in contact_note_cols list from the dataframe
suhi_10_29_2024_df['combined_notes'] = suhi_10_29_2024_df[contact_note_cols].apply(lambda x: ', '.join(x.dropna().astype(str)), axis=1)

In [ ]:
suhi_10_29_2024_df['combined_notes'].value_counts()

,count
combined_notes,
,1079
closed case,9
DECLINED,5
declined services,4
declined services,4
...,...
Assisted pt with scheduling ride to dr appt. Also informed pt about Superior Transport service and provided pt with contact information. Pt stated he has a case manager and has no other needs at this time.,1
"left a vm, Closed Case",1
"The patients phone went straight to voicemail., I reached out to the pt but there was no response. , I made a final call attempt to the pt but there was no response.",1


In [ ]:
# Save this df into a file avoid duplicate index
suhi_10_29_2024_df.to_csv('SUHI_MATRIX_WITH_CHW_Notes_10_29_2024_with_combined_notes.csv', index=False)

In [ ]:
# Read combined notes and phi-3.5 generated matrix
combined_notes_df = pd.read_csv("/content/SUHI_MATRIX_WITH_CHW_Notes_10_29_2024_with_combined_notes.csv")

generated_first_1400_matrix_df = pd.read_csv("/content/generated_df_10_29_2024_v1_till_1400.csv")

generated_after_1400_matrix_df = pd.read_csv("/content/generated_df_10_29_2024_v1_after_1400.csv")

In [ ]:
# Merge first 1400 and after 1400
generated_matrix_df = pd.concat([generated_first_1400_matrix_df, generated_after_1400_matrix_df])

In [ ]:
generated_matrix_df.reset_index(drop=True, inplace=True)

In [ ]:
generated_matrix_df.rename(columns={'0': 'generated_content'}, inplace=True)

In [ ]:
generated_content_dict_list = []
for record in range(len(generated_matrix_df)):
  record_generated_content = generated_matrix_df.iloc[record]['generated_content']
  json_start = record_generated_content.index('{')
  json_end = record_generated_content.rindex('}') + 1
  json_part = record_generated_content[json_start:json_end]
  data_dict = json.loads(json_part)
  generated_content_dict_list.append(data_dict)

In [ ]:
generated_topic_df = pd.DataFrame(generated_content_dict_list)

In [ ]:
generated_topic_df.isna().sum()

,0
Food security,0
Transportation,0
Housing,0
Utilities payment assistance,0
Health Insurance,0
HIV test,0
Covid Vaccine,0
Alcohol,0
Substance,0
Health Education,0


In [ ]:
generated_topic_df['Alcohol'].fillna(generated_topic_df['Alchohol'], inplace=True)
generated_topic_df['reason'].fillna(generated_topic_df['Reason'], inplace=True)

In [ ]:
generated_topic_df

,Food security,Transportation,Housing,Utilities payment assistance,Health Insurance,HIV test,Covid Vaccine,Alcohol,Substance,Health Education,...,Primary Care Physician (PCP),Employment,Comorbidities,Hypertension,Diabetes,Asthma,Patient has any unmet needs,reason,Alchohol,Reason
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"The patient note provided is 'nan', which mean...",0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"The patient note provided is 'nan', which mean...",0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,The patient note only contains the word 'decli...,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,The provided patient note does not contain any...,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,"The patient note provided is 'nan', which mean...",0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615,0,0,0,0,1,0,0,0,0,0,...,1,0,1,0,0,1,1,The patient notes indicate a positive engageme...,0,0
2616,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,The patient note does not contain any explicit...,0,0
2617,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,1,The patient expressed a need for transportatio...,0,0
2618,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,The patient note explicitly mentions 'declined...,0,0


In [ ]:
generated_topic_df.fillna(0, inplace=True)

In [ ]:
generated_topic_df.drop(columns=['Alchohol', 'Reason','reason'], inplace=True)

In [ ]:
generated_topic_df = generated_topic_df.astype(str)

In [ ]:
# generated_topic_df.replace('3', '0', inplace=True)

In [ ]:
generated_topic_df_encoded = pd.get_dummies(generated_topic_df).astype(int)
generated_topic_df_encoded

,Food security_0,Food security_1,Food security_2,Transportation_0,Transportation_1,Transportation_2,Housing_0,Housing_1,Housing_2,Utilities payment assistance_0,...,Hypertension_2,Diabetes_0,Diabetes_1,Diabetes_2,Asthma_0,Asthma_1,Asthma_2,Patient has any unmet needs_0,Patient has any unmet needs_1,Patient has any unmet needs_2
0,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
1,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
3,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
4,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,0,1,0,0,1,0
2616,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0
2617,1,0,0,0,1,0,1,0,0,1,...,0,1,0,0,1,0,0,0,1,0
2618,1,0,0,1,0,0,1,0,0,1,...,0,1,0,0,1,0,0,1,0,0


In [ ]:
for column in generated_topic_df:
  print(generated_topic_df[column].value_counts())

Food security
0    2490
1      61
3      57
2      12
Name: count, dtype: int64
Transportation
0    2454
1      79
3      53
2      34
Name: count, dtype: int64
Housing
0    2461
3     101
1      50
2       8
Name: count, dtype: int64
Utilities payment assistance
0    2510
3      55
1      47
2       8
Name: count, dtype: int64
Health Insurance
0    2486
3      51
1      48
2      35
Name: count, dtype: int64
HIV test
0    2569
3      44
1       5
2       2
Name: count, dtype: int64
Covid Vaccine
0    2529
3      44
1      35
2      12
Name: count, dtype: int64
Alcohol
0    2566
3      43
2      10
1       1
Name: count, dtype: int64
Substance
0    2561
3      43
2      12
1       4
Name: count, dtype: int64
Health Education
0    2525
3      48
1      44
2       3
Name: count, dtype: int64
Emotional needs
0    2426
2      78
1      67
3      49
Name: count, dtype: int64
Domestic Violence
0    2601
3      14
2       4
1       1
Name: count, dtype: int64
Depression
0    2519
2      50
3 

In [ ]:
# Merge existing dataframe with generated notes matrix
combined_notes_generated_df = pd.concat([combined_notes_df, generated_matrix_df, generated_topic_df], axis=1)
combined_notes_generated_df.to_csv('combined_notes_10_29_2024_generated_matrix_df_v1.csv', index=False)

In [ ]:
# Merge existing dataframe with generated notes matrix encoded
combined_notes_generated_df_encoded = pd.concat([combined_notes_df, generated_matrix_df, generated_topic_df_encoded], axis=1)
combined_notes_generated_df_encoded.to_csv('combined_notes_10_29_2024_generated_matrix_df_v1_encoded.csv', index=False)

In [ ]:
combined_notes_generated_df_encoded

,record_id,age,sum_contact_attempts_calc,contact_attempts,day_readmit,total_time_spent,Visit_Number,Count_of_visit,comorbidity,sumComorbidities,...,Hypertension_2,Diabetes_0,Diabetes_1,Diabetes_2,Asthma_0,Asthma_1,Asthma_2,Patient has any unmet needs_0,Patient has any unmet needs_1,Patient has any unmet needs_2
0,2,66.0,NaN,3.0,0.0,0.0,1,2,1,1.0,...,0,1,0,0,1,0,0,1,0,0
1,2,NaN,NaN,3.0,0.0,21.0,2,2,0,0.0,...,0,1,0,0,1,0,0,1,0,0
2,4,50.0,NaN,3.0,1.0,21.0,1,1,0,0.0,...,0,1,0,0,1,0,0,1,0,0
3,5,61.0,NaN,4.0,0.0,50.0,1,2,0,0.0,...,0,1,0,0,1,0,0,1,0,0
4,5,59.0,NaN,3.0,0.0,15.0,2,2,1,1.0,...,0,1,0,0,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2615,2311,60.0,3.0,3.0,0.0,21.0,1,1,0,0.0,...,0,1,0,0,0,1,0,0,1,0
2616,2312,35.0,5.0,3.0,0.0,21.0,1,1,1,1.0,...,0,1,0,0,1,0,0,1,0,0
2617,2313,49.0,NaN,3.0,0.0,21.0,1,1,1,1.0,...,0,1,0,0,1,0,0,0,1,0
2618,2314,62.0,NaN,3.0,0.0,21.0,1,1,0,0.0,...,0,1,0,0,1,0,0,1,0,0
